<a href="https://colab.research.google.com/github/greyhound101/Multihead_attention/blob/master/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! apt install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [2]:
! pip uninstall -y opencv-contrib-python

Found existing installation: opencv-contrib-python 3.4.2.17
Uninstalling opencv-contrib-python-3.4.2.17:
  Successfully uninstalled opencv-contrib-python-3.4.2.17


In [3]:
! pip uninstall -y opencv

In [4]:
! pip install opencv-python==3.4.2.17

In [5]:
! pip install opencv-contrib-python==3.4.2.17

  Using cached opencv_contrib_python-3.4.2.17-cp37-cp37m-manylinux1_x86_64.whl (30.6 MB)


In [6]:
! pip install pymilvus

In [7]:
pip install albumentations==1.0.3

In [8]:
import os
import numpy as np
from PIL import Image
import cv2
from PIL import Image
import base64
from io import BytesIO as _BytesIO
import io
import math

NOR_X = 512
NOR_Y = 384

def calc_sift(image_o):
    sift = cv2.xfeatures2d.SIFT_create()
    # sift  = cv2.ORB_create(nfeatures=200)
    image = cv2.resize(image_o, (NOR_X, NOR_Y))
    if image.ndim == 2:
        gray_image = image
    else:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray_image, None)
    
    # apply the Hellinger kernel by first L1-normalizing and taking the
    # square-root
    if not (des is None) and des.size != 0:
        eps=1e-7
        des /= (des.sum(axis=1, keepdims=True) + eps)
        des = np.sqrt(des)
        # print(des.shape)
        # return des.tolist()[:200] # ashok: remove this 200 since it is filtering only 200
        return des # ashok: remove this 200 since it is filtering only 200

def read_gifs(gif_image):
    imgs = []
    im = Image.open(gif_image)
    for i in range(im.n_frames):
        im.seek(i)
        imgs.append(np.array(im))
    return imgs

def find_duration(img_obj):
    img_obj.seek(0)  # move to the start of the gif, frame 0
    tot_duration = 0
    # run a while loop to loop through the frames
    while True:
        try:
            # returns current frame duration in milli sec.
            frame_duration = img_obj.info['duration']
            tot_duration += frame_duration
            # now move to the next frame of the gif
            img_obj.seek(img_obj.tell() + 1)  # image.tell() = current frame
        except EOFError:
            return tot_duration/1000  # this will return the tot_duration of the gif

def calc_sift_for_gif(gif):
    # decoded_bytes = process_gif_bytes(gif_bytes)
    img = Image.open(gif)
    if img.n_frames > 10:
        num_key_frames = round(find_duration(img)) + 1
    else:
        num_key_frames = img.n_frames 
    # print(num_key_frames+1)
    desc = list()
    for i in range(1, num_key_frames):
        #print(img.n_frames // (num_key_frames * (i)))
        img.seek(img.n_frames // (num_key_frames * (i)))
        image_rgb = img.convert('RGB')
        img_np = np.array(image_rgb)
        image_gray = rgb_to_gray(img_np)
        desc.extend(calc_sift(image_gray))
    unique_desc = [list(x) for x in set(tuple(x) for x in desc)]
    return unique_desc

def calc_sift_for_mp4(mp4):
    cap = cv2.VideoCapture(mp4)
    frameRate = cap.get(5) #frame rate
    x=1
    desc = list()
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            image_gray = rgb_to_gray(frame)
            desc.extend(calc_sift(image_gray))
            x= x+1
    cap.release()
    unique_desc = [list(x) for x in set(tuple(x) for x in desc)]
    return unique_desc  

def calc_sift_for_image(image):
    image_gray = rgb_to_gray(image)
    desc = calc_sift(image_gray)
    return desc

def rgb_to_gray(img):
    if img.ndim == 2:
        gray_image = img
    else:
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_image

def read_image(image_path):
    cv2.imread(image_path)

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [10]:
from joblib import Parallel, delayed
from datetime import datetime
import glob
import os
import json
from tqdm.notebook import tqdm

In [11]:

def calculateMatches(des1,des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    topResults1 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults1.append([m])
            
    matches = bf.knnMatch(des2,des1,k=2)
    topResults2 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults2.append([m])
    
    topResults = []
    for match1 in topResults1:
        match1QueryIndex = match1[0].queryIdx
        match1TrainIndex = match1[0].trainIdx

        for match2 in topResults2:
            match2QueryIndex = match2[0].queryIdx
            match2TrainIndex = match2[0].trainIdx

            if (match1QueryIndex == match2TrainIndex) and (match1TrainIndex == match2QueryIndex):
                topResults.append(match1)
    return topResults

def calculateScore(keypoint1,keypoint2):
    return 100 * (len(calculateMatches(keypoint1,keypoint2))/min(len(keypoint1),len(keypoint2)))


In [19]:
from albumentations import Compose, Blur ,  CenterCrop,RandomScale,ColorJitter
import cv2
from matplotlib import pyplot as plt
cc = Compose([CenterCrop(200, 200)])
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

blur = Compose([Blur(blur_limit=10,p=1)])
scl = Compose([RandomScale (scale_limit=0.5, interpolation=1,p=1)])
ls=[]
cj = Compose([ColorJitter  (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1)])
from tqdm import tqdm
import glob
for path in tqdm(list(glob.glob('/content/gdrive/MyDrive/Rarible_Images/*.png'))[:10]):
  a=cv2.imread(path )
  org=calc_sift_for_image(a)
  ls.append(calculateScore(org,calc_sift_for_image(cc(image=a)['image'])))
  ls.append(calculateScore(org,calc_sift_for_image(blur(image=a)['image'])))
  ls.append(calculateScore(org,calc_sift_for_image(scl(image=a)['image'])))


  for angle in range(-100,100,50):
    ls.append(calculateScore(org,calc_sift_for_image(rotate_image(a,angle))))



100%|██████████| 10/10 [00:04<00:00,  2.41it/s]


In [20]:
np.mean(ls)

44.89448221650154